In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [3]:
from src.mlflow_demo.constants import *
from src.mlflow_demo.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        create_directories([self.config.data_ingestion.root_dir])
        return DataIngestionConfig(
            root_dir=self.config.data_ingestion.root_dir,
            source_URL=self.config.data_ingestion.source_URL,
            local_data_file=self.config.data_ingestion.local_data_file,
            unzip_dir=self.config.data_ingestion.unzip_dir
        )

In [6]:
import os
import zipfile
from src.mlflow_demo import logger
import urllib.request as req
from src.mlflow_demo.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = req.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f'File {filename} downloaded with info {headers}')
        else:
            logger.info(f'File {self.config.local_data_file} already exists with size {get_size(Path(self.config.local_data_file))}')
            
    def extract_zip_file(self):
        os.makedirs(self.config.unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip:
            zip.extractall(self.config.unzip_dir)


In [10]:
# Pipeline - All toghether

try:
    config_manager = ConfigurationManager()
    config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-30 17:31:52,069: INFO: common: Yaml config/config.yml loaded successfully.]
[2023-07-30 17:31:52,072: INFO: common: Yaml params.yml loaded successfully.]
[2023-07-30 17:31:52,074: INFO: common: Yaml schema.yml loaded successfully.]
[2023-07-30 17:31:52,076: INFO: common: created directory: artifacts]
[2023-07-30 17:31:52,078: INFO: common: created directory: artifacts/data_ingestion]
[2023-07-30 17:31:52,793: INFO: 918654868: File artifacts/data_ingestion/data.zip downloaded with info Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 95D0:0962:272EDD3:28D063C:64C67459
Accept-Ranges: bytes
Date: Sun, 30 Jul 2023 14:31:53 GMT
Via: 1.1